In [ ]:
!pip install transformers datasets peft accelerate optuna wandb evaluate scikit-learn -q


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.9/400.9 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.0/247.0 kB 24.4 MB/s eta 0:00:00


In [ ]:
import os
import torch
import numpy as np
import evaluate
import optuna
import wandb
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer
)
from peft import IA3Config, get_peft_model, TaskType

wandb.login()


wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


True

In [ ]:

data_path_prefix = "/content/drive/MyDrive/Banking77_Project/data/"
data_files = {
    "train": os.path.join(data_path_prefix, "train.csv"),
    "validation": os.path.join(data_path_prefix, "validation.csv"),
    "test": os.path.join(data_path_prefix, "test.csv"),
}

dataset = load_dataset('csv', data_files=data_files)

label_names = sorted(list(set(dataset["train"]["intent"])))
label2id = {label: idx for idx, label in enumerate(label_names)}
id2label = {idx: label for label, idx in label2id.items()}
num_labels = len(label_names)


In [ ]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['text', 'intent', 'text_cleaned'],
        num_rows: 8002
    })
    validation: Dataset({
        features: ['text', 'intent', 'text_cleaned'],
        num_rows: 2001
    })
    test: Dataset({
        features: ['text', 'intent', 'text_cleaned'],
        num_rows: 3080
    })
})


In [ ]:
model_name = "roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def preprocess(example):
    return tokenizer(example["text"], truncation=True, padding="max_length", max_length=64)

dataset = dataset.map(preprocess)
dataset = dataset.map(lambda e: {"label": [label2id[l] for l in e["intent"]]}, batched=True)


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

...

Map:   0%|          | 0/2001 [00:00<?, ? examples/s]

Map:   0%|          | 0/3080 [00:00<?, ? examples/s]

Map:   0%|          | 0/8002 [00:00<?, ? examples/s]

Map:   0%|          | 0/2001 [00:00<?, ? examples/s]

Map:   0%|          | 0/3080 [00:00<?, ? examples/s]

In [ ]:
accuracy = evaluate.load("accuracy")
f1 = evaluate.load("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    return {
        "accuracy": accuracy.compute(predictions=preds, references=labels)["accuracy"],
        "f1": f1.compute(predictions=preds, references=labels, average="weighted")["f1"]
    }


In [ ]:
def objective(trial):
    # Suggest hyperparams
    learning_rate = trial.suggest_float("learning_rate", 1e-5, 5e-4, log=True)
    batch_size = trial.suggest_categorical("batch_size", [16, 32, 64])
    num_epochs = trial.suggest_int("num_train_epochs", 3, 7)
    weight_decay = trial.suggest_float("weight_decay", 0.0, 0.1)

    # W&B init per trial
    wandb.init(
        project="banking77-ia3-optuna",
        name=f"trial_{trial.number}",
        reinit=True,
        config={
            "learning_rate": learning_rate,
            "batch_size": batch_size,
            "num_epochs": num_epochs,
            "weight_decay": weight_decay
        }
    )

    # Build model
    base_model = AutoModelForSequenceClassification.from_pretrained(
        model_name,
        num_labels=num_labels,
        id2label=id2label,
        label2id=label2id
    )
    ia3_config = IA3Config(task_type=TaskType.SEQ_CLS, target_modules=["query", "value"])
    model = get_peft_model(base_model, ia3_config)

    # Training args
    training_args = TrainingArguments(
        output_dir=f"./results/trial_{trial.number}",
        eval_strategy="epoch",
        save_strategy="no",
        learning_rate=learning_rate,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        num_train_epochs=num_epochs,
        weight_decay=weight_decay,
        logging_dir="./logs",
        report_to=["wandb"]
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=dataset["train"],
        eval_dataset=dataset["validation"],
        tokenizer=tokenizer,
        compute_metrics=compute_metrics,
    )

    trainer.train()
    eval_results = trainer.evaluate()

    wandb.log(eval_results)
    wandb.finish()

    return eval_results["eval_accuracy"]


In [ ]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=10)

print("Best Trial:")
print(study.best_trial.params)


[I 2025-08-22 08:51:10,512] A new study created in memory with name: no-name-98355eb9-d9f8-4e7a-94c5-db16d17b554d


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipython-input-1123742093.py:45: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,4.277600,4.131026,0.068966,0.038914
2,3.716000,2.934323,0.490255,0.448305
3,2.392600,1.702281,0.640680,0.614771
4,1.655500,1.310027,0.708146,0.693115
5,1.423700,1.219900,0.717141,0.704071


epoch,▁
eval/accuracy,▁▆▇███
eval/f1,▁▅▇███
eval/loss,█▅▂▁▁▁
eval/runtime,▁▃▇▆▇█
eval/samples_per_second,█▆▂▃▂▁
eval/steps_per_second,█▆▂▃▂▁
eval_accuracy,▁
eval_f1,▁
eval_loss,▁
eval_runtime,▁


[I 2025-08-22 08:56:40,307] Trial 0 finished with value: 0.7171414292853573 and parameters: {'learning_rate': 0.0001730151117875226, 'batch_size': 16, 'num_train_epochs': 5, 'weight_decay': 0.0681725169429884}. Best is trial 0 with value: 0.7171414292853573.


...

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipython-input-1123742093.py:45: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,4.316200,4.277278,0.037481,0.018536
2,4.257200,4.218113,0.071964,0.025208
3,4.190900,4.156759,0.071464,0.041655
4,4.141900,4.131039,0.093953,0.063004


epoch,▁
eval/accuracy,▁▅▅██
eval/f1,▁▂▅██
eval/loss,█▅▂▁▁
eval/runtime,▁█▁▇▂
eval/samples_per_second,█▁█▂▇
eval/steps_per_second,█▁█▂▇
eval_accuracy,▁
eval_f1,▁
eval_loss,▁
eval_runtime,▁


[I 2025-08-22 09:48:33,266] Trial 9 finished with value: 0.09395302348825588 and parameters: {'learning_rate': 7.613237902003231e-05, 'batch_size': 16, 'num_train_epochs': 4, 'weight_decay': 0.0422396494741149}. Best is trial 1 with value: 0.7391304347826086.


Best Trial:
{'learning_rate': 0.00013217439430404193, 'batch_size': 16, 'num_train_epochs': 7, 'weight_decay': 0.010321813929659529}


In [ ]:
best_params = study.best_trial.params

base_model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=num_labels,
    id2label=id2label,
    label2id=label2id
)
ia3_config = IA3Config(task_type=TaskType.SEQ_CLS, target_modules=["query", "value"])
model = get_peft_model(base_model, ia3_config)

final_args = TrainingArguments(
    output_dir="./final_results",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=best_params["learning_rate"],
    per_device_train_batch_size=best_params["batch_size"],
    per_device_eval_batch_size=best_params["batch_size"],
    num_train_epochs=best_params["num_train_epochs"],
    weight_decay=best_params["weight_decay"],
    load_best_model_at_end=True,
    logging_dir="./logs_final",
    report_to=["wandb"]
)

final_trainer = Trainer(
    model=model,
    args=final_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

final_trainer.train()
results = final_trainer.evaluate(dataset["test"])
print("Final Test Results:", results)


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipython-input-726709876.py:26: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  final_trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,4.295700,4.204616,0.044478,0.014498
2,4.016000,3.641601,0.372314,0.330674
3,3.108200,2.353835,0.554723,0.517513
4,2.094500,1.582701,0.650175,0.626654
5,1.598500,1.273890,0.704648,0.691441
6,1.381700,1.140340,0.727636,0.716337
7,1.291300,1.103214,0.734633,0.723760


Final Test Results: {'eval_loss': 1.1605722904205322, 'eval_accuracy': 0.7253246753246754, 'eval_f1': 0.7075864702367034, 'eval_runtime': 10.192, 'eval_samples_per_second': 302.199, 'eval_steps_per_second': 18.937, 'epoch': 7.0}
